## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-05-06-52-50 +0000,nyt,‘A Big Positive’: How One Company Plans to Pro...,https://www.nytimes.com/2025/11/03/health/medi...
1,2025-11-05-06-49-00 +0000,wsj,"For Democrats, Tuesday brought more than three...",https://www.wsj.com/politics/elections/democra...
2,2025-11-05-06-48-00 +0000,wsj,"Siemens Healthineers Reports Fall in Profits, ...",https://www.wsj.com/business/earnings/siemens-...
3,2025-11-05-06-43-49 +0000,nypost,"Strange swirling light spotted over NYC, Long ...",https://nypost.com/2025/11/05/us-news/strange-...
4,2025-11-05-06-40-30 +0000,nyt,Scars in the Blue Mosque Reveal Afghanistan’s ...,https://www.nytimes.com/2025/11/05/world/afric...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2448/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
24,trump,47
70,new,32
11,election,23
274,budget,21
137,mamdani,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
8,2025-11-05-06-34-00 +0000,wsj,The Democratic Party won the New York City may...,https://www.wsj.com/politics/elections/five-ta...,220
393,2025-11-04-09-30-53 +0000,bbc,Trump backs Cuomo for New York City mayor and ...,https://www.bbc.com/news/articles/c231e284345o...,156
45,2025-11-05-04-42-00 +0000,wsj,"Andrew Cuomo, part of New York’s political fir...",https://www.wsj.com/politics/elections/half-a-...,143
232,2025-11-04-21-01-07 +0000,nyt,"In New Jersey, Democrats and Republicans say T...",https://www.nytimes.com/live/2025/11/04/us/ele...,141
391,2025-11-04-10-00-00 +0000,wsj,When the Supreme Court hears arguments over Pr...,https://www.wsj.com/politics/policy/whats-at-s...,139


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
8,220,2025-11-05-06-34-00 +0000,wsj,The Democratic Party won the New York City may...,https://www.wsj.com/politics/elections/five-ta...
45,103,2025-11-05-04-42-00 +0000,wsj,"Andrew Cuomo, part of New York’s political fir...",https://www.wsj.com/politics/elections/half-a-...
391,73,2025-11-04-10-00-00 +0000,wsj,When the Supreme Court hears arguments over Pr...,https://www.wsj.com/politics/policy/whats-at-s...
400,68,2025-11-04-09-00-00 +0000,cbc,Carney's 1st budget calls for billions in new ...,https://www.cbc.ca/news/politics/carney-first-...
31,68,2025-11-05-05-33-00 +0000,wsj,Lawmakers are hopeful they can reach a biparti...,https://www.wsj.com/politics/policy/lawmakers-...
346,62,2025-11-04-12-59-47 +0000,wapo,"Dick Cheney, powerful vice president during wa...",https://www.washingtonpost.com/obituaries/2025...
102,55,2025-11-05-02-17-39 +0000,nypost,Virginia Governor-elect Abigail Spanberger’ ca...,https://nypost.com/2025/11/04/us-news/virginia...
379,52,2025-11-04-11-00-00 +0000,latimes,Elon Musk's latest Tesla pay package could mak...,https://www.latimes.com/business/story/2025-11...
326,45,2025-11-04-15-01-47 +0000,nypost,France threatens to block Shein over sale of c...,https://nypost.com/2025/11/04/world-news/franc...
212,45,2025-11-04-21-33-00 +0000,wsj,Amgen Raises Full-Year Outlook After Third-Qua...,https://www.wsj.com/business/earnings/amgen-ra...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
